# Visual Plant Recognition

This Jupyter Notebook uses a pre-trained Inception v3 architecture to train on the [Oxford 102 dataset](https://www.robots.ox.ac.uk/~vgg/data/flowers/102/). The dataset consists of 102 different plants. PyTorch is used as a framework.

## Dataset Loading

Importing all the necessary frameworks, libraries and classes

In [ ]:
from pathlib import Path
import copy
from matplotlib import pyplot as plt
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import platform 

from helper import convert_mat_to_csv, counting_labels
from dataset_loader import Oxford102Dataset, Rescale, RandomCrop, ToTensor, Normalize, per_image_weights
from visualization import image_visualization, batch_visualization, class_balance_visualization, per_class_accuracy, worst_classes
from model import SimpleNet, resNet_152
from training import train_epoch
from evaluation import evaluation

In [ ]:
#print(platform.platform())
#print(torch.has_mps)

Converting the MatLab files with the labels into a csv

In [ ]:
path_dataset = Path("../dataset/")
if not(path_dataset.joinpath("imagelabels.csv").exists()):
    convert_mat_to_csv(path_dataset.joinpath("imagelabels.mat"))
    print("Created image labels")
else: 
    print("Image labels already exist as csv")
if not(path_dataset.joinpath("setid.csv").exists()):    
    convert_mat_to_csv(path_dataset.joinpath("setid.mat"))
    print("Created set id")
else: 
    print("Set id already exist as csv")

Visualization of the dataset

In [ ]:
oxford102Dataset = Oxford102Dataset(csv_file='../dataset/imagelabels.csv',
                                    dataset_dir='../dataset/')

batch_visualization(oxford102Dataset, (1, 5), (15, 4))
    


Checking for class balance

In [ ]:
class_sample_counts = counting_labels(filename='../dataset/imagelabels.csv')
class_balance_visualization(class_sample_counts)

In [ ]:
scale = Rescale(330)
crop = RandomCrop(299)
composed = transforms.Compose([Rescale(330),
                               RandomCrop(299)])

# Apply each of the above transforms on sample.
fig = plt.figure()
sample = oxford102Dataset[65]
for i, tsfrm in enumerate([scale, crop, composed]):
    transformed_sample = tsfrm(sample)
    image_visualization(transformed_sample['image'], sample['plant_label'])


plt.show()

Putting everything together and iterating trough the dataset with dataloader

Loading the Oxford 102 Dataset with a custome dataloader and visualise it

In [ ]:
transformed_dataset = Oxford102Dataset(csv_file='../dataset/imagelabels.csv',
                                    dataset_dir='../dataset/', transform=transforms.Compose([
        RandomCrop(224),
        ToTensor(),
        Normalize()]))

train_len = int(0.8 * len(transformed_dataset))
train_data, test_data = random_split(transformed_dataset, [train_len, 
                                                           len(transformed_dataset)-train_len], 
                                                             generator=torch.Generator().manual_seed(1))

weights = per_image_weights(oxford102Dataset, class_sample_counts)
weights = torch.tensor(weights)                                       
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_dataloader = DataLoader(train_data, batch_size=32,
                         num_workers=4, sampler = sampler)
test_dataloader = DataLoader(test_data, batch_size=32,
                        shuffle=True, num_workers=4)

## Training

In [ ]:
# instanciate model
model = SimpleNet()

# move model to GPU if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model.cuda()

# move model to Apple Silicon GPU if available
#device = torch.device('mps')
#if device:
   # model.to(device)
#device
    
# loss function
loss_function = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)

In [ ]:
def train(n_epochs, train_dataset, model, optimizer, loss_function, use_cuda, use_mps, batch_size=32, k_splits = 5):
    # tensorboard
    writer = SummaryWriter()

    # cross validation
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    kfold = KFold(n_splits=k_splits, shuffle=True, random_state=1)
    for fold, (train_idx, test_idx) in enumerate(kfold.split(np.arange(len(train_dataset)))):
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        valid_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_subsampler)
        valid_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=valid_subsampler)
        
        model.apply(init_weights)

        train_eval = evaluation(model, train_dataloader, loss_function, use_cuda, use_mps)
        validation_eval = evaluation(model, valid_dataloader, loss_function, use_cuda, use_mps)
        writer.add_scalars(f'Loss/fold: {fold + 1}', {'training': train_eval[0], 'validation': validation_eval[0]}, global_step=0)
        writer.add_scalars(f'Accuracy/fold: {fold + 1}', {'training': train_eval[1], 'validation': validation_eval[1]}, global_step=0)

        # Visualization
        fig = per_class_accuracy(train_eval[2])
        writer.add_figure(f'Accuracy per class', fig, global_step=0)
        fig = worst_classes(train_eval[2], 5)
        writer.add_figure(f'Worst classes', fig, global_step=0)

        for epoch in range(1, n_epochs+1):
            train_epoch(model, train_dataloader, optimizer, loss_function, use_cuda, use_mps)
            # evaluation
            train_eval = evaluation(model, train_dataloader, loss_function, use_cuda, use_mps)
            validation_eval = evaluation(model, valid_dataloader, loss_function, use_cuda, use_mps)
            writer.add_scalars(f'Loss/fold: {fold + 1}', {'training': train_eval[0], 'validation': validation_eval[0]}, global_step=epoch)
            writer.add_scalars(f'Accuracy/fold: {fold + 1}', {'training': train_eval[1], 'validation': validation_eval[1]}, global_step=epoch)

            # Visualization
            fig = per_class_accuracy(train_eval[2])
            writer.add_figure(f'Accuracy per class', fig, global_step=epoch)
            fig = worst_classes(train_eval[2], 5)
            writer.add_figure(f'Worst classes', fig, global_step=epoch)
            
            print(f"Split {fold + 1}, Epoch {epoch} - Loss: {train_eval[0]} - Acc: {train_eval[1]} - ValidLoss: {validation_eval[0]} - ValidAcc {validation_eval[1]}")
            
        validation_eval = evaluation(model, valid_dataloader, loss_function, use_cuda)
        if  validation_eval[1] > best_acc:
                best_acc = validation_eval[1]
                best_model_wts = copy.deepcopy(model.state_dict())
                print("Saved model")
    return model


In [ ]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

In [ ]:
# train the model
model_scratch = train(15, train_data, model, optimizer, 
                      loss_function, use_cuda, use_mps=False, k_splits=2)

In [ ]:
model_resnet152 = resNet_152(1000, 7)
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_resnet152.cuda()

# move model to Apple Silicon GPU if available
#device = torch.device('mps')
#if device:
    #model_resnet152.to(device)
#device

In [ ]:
model_scratch = train(5, train_data, model, optimizer, 
                      loss_function, use_cuda, use_mps=False, k_splits=2)